# AWS DynamoDB

>[Amazon AWS DynamoDB](https://awscli.amazonaws.com/v2/documentation/api/latest/reference/dynamodb/index.html) 是一个完全托管的 `NoSQL` 数据库服务，可提供快速且可预测的性能以及无缝的可扩展性。

本笔记本将介绍如何使用 `DynamoDB` 通过 `DynamoDBChatMessageHistory` 类存储聊天消息历史记录。

## 设置

首先，请确保您已正确配置 [AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html)。然后，请确保您已安装 `langchain-community` 包，我们需要安装它。同时，我们还需要安装 `boto3` 包。

```bash
pip install -U langchain-community boto3
```

在 LangSmith 上设置好 LangSmith（https://smith.langchain.com/）以获得一流的可观测性也是有益的（但并非必需）。

In [ ]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [ ]:
from langchain_community.chat_message_histories import (
    DynamoDBChatMessageHistory,
)

## 创建表

现在，创建我们将用于存储消息的 `DynamoDB` 表：

In [10]:
import boto3

# Get the service resource.
dynamodb = boto3.resource("dynamodb")

# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName="SessionTable",
    KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
    AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
    BillingMode="PAY_PER_REQUEST",
)

# Wait until the table exists.
table.meta.client.get_waiter("table_exists").wait(TableName="SessionTable")

# Print out some data about the table.
print(table.item_count)

0


## DynamoDBChatMessageHistory

In [11]:
history = DynamoDBChatMessageHistory(table_name="SessionTable", session_id="0")

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [5]:
history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

## DynamoDBChatMessageHistory：自定义端点 URL

有时指定连接到 AWS 端点的 URL 会非常有用。例如，当您在本地环境中运行，并通过 [Localstack](https://localstack.cloud/) 连接时。在这些情况下，您可以通过构造函数中的 `endpoint_url` 参数来指定 URL。

In [13]:
history = DynamoDBChatMessageHistory(
    table_name="SessionTable",
    session_id="0",
    endpoint_url="http://localhost.localstack.cloud:4566",
)

## DynamoDBChatMessageHistory 配合复合主键
DynamoDBChatMessageHistory 的默认键是 ```{"SessionId": self.session_id}```，但您可以修改它以匹配您的表设计。

### 主键名称
您可以通过在构造函数中传入 `primary_key_name` 值来修改主键，从而得到以下结果：
```{self.primary_key_name: self.session_id}```

### 复合主键
当使用现有的 DynamoDB 表时，您可能需要将键结构从默认值修改为包含排序键的内容。为此，您可以使用 `key` 参数。

传递 `key` 参数的值将覆盖 `primary_key` 参数，由此产生的键结构将是传递的值。

In [14]:
composite_table = dynamodb.create_table(
    TableName="CompositeTable",
    KeySchema=[
        {"AttributeName": "PK", "KeyType": "HASH"},
        {"AttributeName": "SK", "KeyType": "RANGE"},
    ],
    AttributeDefinitions=[
        {"AttributeName": "PK", "AttributeType": "S"},
        {"AttributeName": "SK", "AttributeType": "S"},
    ],
    BillingMode="PAY_PER_REQUEST",
)

# Wait until the table exists.
composite_table.meta.client.get_waiter("table_exists").wait(TableName="CompositeTable")

# Print out some data about the table.
print(composite_table.item_count)

0


In [8]:
my_key = {
    "PK": "session_id::0",
    "SK": "langchain_history",
}

composite_key_history = DynamoDBChatMessageHistory(
    table_name="CompositeTable",
    session_id="0",
    endpoint_url="http://localhost.localstack.cloud:4566",
    key=my_key,
)

composite_key_history.add_user_message("hello, composite dynamodb table!")

composite_key_history.messages

[HumanMessage(content='hello, composite dynamodb table!')]

## 链式调用

我们可以轻松地将此消息历史类与 [LCEL Runnables](/docs/how_to/message_history) 结合使用。

为此，我们将使用 OpenAI，因此我们需要安装它。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatOpenAI()

In [ ]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=session_id
    ),
    input_messages_key="question",
    history_messages_key="history",
)

In [ ]:
# This is where we configure the session id
config = {"configurable": {"session_id": "<SESSION_ID>"}}

In [9]:
chain_with_history.invoke({"question": "Hi! I'm bob"}, config=config)

AIMessage(content='Hello Bob! How can I assist you today?')

In [10]:
chain_with_history.invoke({"question": "Whats my name"}, config=config)

AIMessage(content='Your name is Bob! Is there anything specific you would like assistance with, Bob?')